# Create LM

In [1]:
import httpx
from typing import List, Optional
import brokit as bk
class Llama(bk.LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=bk.ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[bk.Message]]=None, images:Optional[List[bk.Image]]=None, audios=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, bk.Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        if images:
            _messages[-1]["images"] = [image.to_base64() for image in images]
        request_params = {
            "model": self.model_name,
            "messages": _messages,
            "stream": False,
            "options": {**params},
        }
        response = self.client.post(
            url,
            json=request_params
        )                
        return response.json()

    def response(self, original_response: dict) -> bk.ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return bk.ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=bk.Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

In [2]:
class QA(bk.Prompt):
    """Answer a question in bro-like manner"""
    question:str = bk.InputField(description="a user's question")
    answer:str = bk.OutputField(description="answer to the question")

llama = Llama(model_name="gemma3:12b")
qa = bk.Predictor(prompt=QA, lm=llama)

In [3]:
mock_fn1 = lambda: {"response":True}
mock_fn2 = lambda: {"response":False}

class ProgQA(bk.Program):
    def __init__(self):
        self.qa = bk.Predictor(prompt=QA, lm=llama)
    
    def forward(self, question: str) -> dict:
        self.step("check1", mock_fn1)
        self.step("check2", mock_fn2)
        answer = self.step("qa", self.qa, question=question)
        return answer.answer

In [4]:
pqa = ProgQA()

In [5]:
pqa(question="Hi")

"Yo! What's up, my dude? Just hit me with whatever you got. Let's do this."

In [6]:
pqa.history

[ProgramHistory(id='60d4f3ec-071e-4958-9bc4-f8b43acea774', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 44, 90761), error=None, response_ms=0.0017999991541728377, step_name='check1', component_type='custom', inputs={}, outputs={'response': True}, child_call_ids=[]),
 ProgramHistory(id='4cd008d8-18d9-4615-8aed-b52be86c571d', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 44, 90761), error=None, response_ms=0.0005000001692678779, step_name='check2', component_type='custom', inputs={}, outputs={'response': False}, child_call_ids=[]),
 ProgramHistory(id='9f6701c3-563f-475f-9d2c-4de8fe3ef256', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 47, 156727), error=None, response_ms=3065.9875999999713, step_name='qa', component_type='Predictor', inputs={'question': 'Hi'}, outputs={'answer': "Yo! What's up, my dude? Just hit me with whatever you got. Let's do this."}, child_call_ids=['4a79b98e-9cd1-45e9-8eb4-25b7d1aea197'])]

In [7]:
pqa.qa.history

[PredictorHistory(id='4a79b98e-9cd1-45e9-8eb4-25b7d1aea197', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 47, 156727), error=None, response_ms=3065.8680000005916, predictor_name='QA', inputs=[{'role': 'system', 'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in bro-like manner"}, {'role': 'user', 'content': '<||question||>\nHi\n\nRespond with the corresponding output fields, starting with the field: `<||answer||>` and then ending with the marker for `<||completed||>`.'}], outputs={'answer': "Yo! What's up, my dude? Just hit me with whatever you got. Let's do this."}, lm_call_id='12b52864-fa55-46d2-b61e-04fc97ea34f7')]

In [8]:
llama.history

[LMHistory(id='12b52864-fa55-46d2-b61e-04fc97ea34f7', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 47, 156727), error=None, response_ms=3065.8036999993783, model_name='gemma3:12b', model_type='chat', request=[{'role': 'system', 'content': "Your input fields are:\n1. question (<class 'str'>): a user's question\nYour output fields are:\n1. answer (<class 'str'>): answer to the question\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question in bro-like manner"}, {'role': 'user', 'content': '<||question||>\nHi\n\nRespond with the corresponding output fields, starting with the field: `<||answer||>` and then ending with the marker for `<||completed||>`.'}], response="<||answer||>\nYo! What's up, my dude? Just hit me with whatever you got. Let's do this.\n\n<||completed||>", usage={'input_tokens': 157,

In [9]:
print(pqa.trace())

1. ✓ check1 (custom)
2. ✓ check2 (custom)
3. ✓ qa (Predictor)


In [10]:
pqa.get_step("qa")

ProgramHistory(id='9f6701c3-563f-475f-9d2c-4de8fe3ef256', timestamp=datetime.datetime(2026, 2, 7, 15, 44, 47, 156727), error=None, response_ms=3065.9875999999713, step_name='qa', component_type='Predictor', inputs={'question': 'Hi'}, outputs={'answer': "Yo! What's up, my dude? Just hit me with whatever you got. Let's do this."}, child_call_ids=['4a79b98e-9cd1-45e9-8eb4-25b7d1aea197'])